In [1]:
import pandas as pd
from scholarly import scholarly, ProxyGenerator
from habanero import Crossref
from pyscopus import Scopus

scopus = Scopus("f130ed2ec5730526ea6c672a96600fd7")

# [Worked] Get Metadata based on Titles from Scopus

In [17]:
titles = pd.read_csv("snowballing_additional_backward.csv", sep=";")
titles.head()

,Title
0,Knowledge-based lifecycle management approach ...
1,Unified IoT ontology to enable interoperabilit...
2,Agile Manifesto
3,Development of a knowledge-based design suppor...
4,Framework for the agile development of innovat...


In [ ]:
ids = []
failed_titles = []
for title in titles["Title"]:
    try:
        result = scopus.search("Title(" + title + ")")
        ids.append(result["scopus_id"])
        print("success")
    except Exception as e:
        print("failed: ", e)
        failed_titles.append(title)

### persist scopus ids

In [20]:
id_test = []
for row in ids:
    if not row.empty:
        id_test.append(row[0])
id_list = pd.Series(id_test)
id_list.to_csv("scopus_ids_additional_backward.csv", sep=";", index=False)
failed_list = pd.Series(failed_titles)
failed_list.to_csv("scopus_failed_backward.csv", sep=";", index=False)

### load scopus ids from previous run, and get metadata for those ids

In [2]:
scopus_ids = pd.read_csv("scopus_ids_additional_backward.csv", sep=";", encoding="latin-1")

In [ ]:
columns = ["title", "abstract", "dc:publisher", "prism:aggregationType", "prism:publicationName", "prism:coverDate", "prism:url"]
result = pd.DataFrame(columns=columns)
failed_ids = []
for index, scopus_id in scopus_ids.iterrows():
    try:
        pub_info = scopus.retrieve_abstract(scopus_id[0])
        new_line = {}
        print("success")
        for column in columns:
            new_line[column] = pub_info[column]
        result = result.append(new_line, ignore_index=True)
    except Exception as ex:
        failed_ids.append(scopus_id)
        print("failed", ex)
        continue
    

In [38]:
pub_info = scopus.retrieve_abstract("85143907362")
new_line = {}
for column in columns:
            new_line[column] = pub_info[column]
result = result.append(new_line, ignore_index=True)

C:\Users\Daniel Lehner\AppData\Local\Temp\ipykernel_14724\3248324592.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_line, ignore_index=True)


## persist metadata in file finally.csv

In [23]:
result.to_csv("finally_additional_backward.csv", sep=";")

In [5]:
len(failed_ids)
pd.Series(failed_ids).to_csv("failed_ids.csv", sep=";", index=False)

## [WORKED] Get DOIs from CrossRef, and store in dois.csv

In [12]:
cr = Crossref()
x = cr.works(query = "A Catalog of Design Patterns for Compositional Language Engineering")

In [ ]:
for title in titles["Title"]:
    print(title)

In [ ]:
for title in titles["Title"]:
    query_results = cr.works(query = title)
    items = query_results['message']['items']
    for res in items:
        #print(res['title'][0])
        if "title" not in res.keys():
            continue
        if res['title'][0].lower() == title.lower():
            print(res["DOI"])
            dois.append(res["DOI"])

In [37]:
doi_list = pd.Series(dois)
#doi_list.to_csv("dois.csv", header="DOI", sep=";")

## [Banned] Get Metadata based on DOIs from Scholarly

In [ ]:
dois = pd.read_csv("dois.csv", sep=";", encoding="latin-1")
dois = pd.DataFrame(dois["DOI"].unique(), columns=["DOI"])

In [5]:
pg = ProxyGenerator()
#pg.Tor_Internal(tor_cmd = "tor")
pg.FreeProxies()
scholarly.use_proxy(pg)

In [6]:
result = scholarly.search_pubs("10.1016/j.cie.2022.108659")
x = next(result)

MaxTriesExceededException: Cannot Fetch from Google Scholar.

In [ ]:
titles_and_abstracts = pd.DataFrame(columns=["Title", "Abstract"])
failed_titles = []
failed_count = 0
success_count = 0
for title in dois['DOI']:
    #pg = ProxyGenerator()
    #pg.FreeProxies()
    #scholarly.use_proxy(pg)
    results = scholarly.search_pubs(title)
    try:
        result = next(results)
    except StopIteration:
        failed_titles.append(title)
        failed_count = failed_count + 1
        print("gotcha:" + str(failed_count))
        continue
    success_count = success_count + 1
    print("success: " + str(success_count))
    new_row = {"Title": result['bib']['title'], "Abstract": result['bib']['abstract']}
    titles_and_abstracts = titles_and_abstracts.append(new_row, ignore_index=True)